In [1]:
import tensorflow as tf
import numpy as np

# tf.train.Example
## tf.Example의 데이터 형식
기본적으로 tf.Example은 {string : tf.train.Feature} 매핑입니다.
tf.train.Feature 메시지 유형은 다음 3가지 유형 중 하나를 허용할 수 있습니다.

### 1. tf.train.ByteList(다음 유형을 강제 변환할 수 있다)
- string
- byte

In [4]:
def _bytes_feature(value):
    # string / byte 로부터 byte_list를 만든다
    # isinstance(변수, 타입) 변수가 타입이 맞는지를 판단해준다. 아래에서는 텐서유형일 경우 변환을 진행하지 않아 넘파이로 변경해주는 작업이다.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list= tf.train.BytesList(value = [value]))

In [8]:
print(_bytes_feature(b'test_string'))
print(_bytes_feature(u'test_string'.encode('utf-8')))

bytes_list {
  value: "test_string"
}

bytes_list {
  value: "test_string"
}



### 2. tf.train.FloatList
- float (float32)
- double (float64)

In [9]:
def _float_feature(value):
    # float과 double를 float_list로 돌려준다.
    return tf.train.Feature(float_list = tf.train.FloatList(value=[value]))

In [29]:
print(type(np.exp(1)))
print(np.exp(1))
k = _float_feature(np.exp(1))
print(k)

<class 'numpy.float64'>
2.718281828459045
float_list {
  value: 2.7182817459106445
}



### 3. tf.train.Int64List
- bool
- enum
- int32
- unit32
- int64
- uint64

In [30]:
def _int64_feature(value):
    # bool / enum / int / unit 를 int64_list로 리턴한다.
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [31]:
print(_int64_feature(True))
print(_int64_feature(1))
print(_int64_feature(2))

int64_list {
  value: 1
}

int64_list {
  value: 1
}

int64_list {
  value: 2
}



In [33]:
feature = _float_feature(np.exp(1))
print(feature)
print(feature.SerializeToString())

float_list {
  value: 2.7182817459106445
}

b'\x12\x06\n\x04T\xf8-@'


### tf.train.Example 메세지 만들기

In [34]:
n_obervaions = int(1e4)

feature0 = np.random.choice([False, True], n_obervaions)
feature1 = np.random.randint(0, 5, n_obervaions)
strings = np.array([b'dog', b'cat', b'chicken', b'puppy', b'sheep'])
feature2 = strings[feature1]
feature3 = np.random.randn(n_obervaions)

In [36]:
def serialize_example(feature0, feature1, feature2, feature3):
    feature = {
        'feature0' : _int64_feature(feature0),
        'feature1' : _int64_feature(feature1), 
        'feature2' : _bytes_feature(feature2),
        'feature3' : _float_feature(feature3)
    }

    example_proto = tf.train.Example(features = tf.train.Features(feature=feature))
    
    return example_proto.SerializeToString()

In [37]:
serialized_ex = serialize_example(False, 4 , b'dog', 0.9876)
serialized_ex

b'\nQ\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x00\n\x13\n\x08feature2\x12\x07\n\x05\n\x03dog\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04[\xd3|?\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x04'

메세지는 tf.train.Example.FromString 메서드를 통해 디코딩할 수 있다.

In [ ]:
ex_proto = tf.train.Exa